In [776]:
import os
import numpy as np
# import models, layers, utils
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# import Sequential
from tensorflow.keras import models, layers, utils
from tensorflow.keras import Sequential
import sounddevice as sd
import re


In [777]:
# Step 1: Data Collection
sound_folder = "hello_split"
words = []
sound_data = []
label = []
name = []
# sound_name = ['Miro.', 'Naam.', 'Saurabh.', 'Baral.','ho.','tapai.','Ko.', 'xa.','I.','love.','you.']

In [778]:
# Iterate over sound files in the folder
i = -1
for filename in os.listdir(sound_folder):
    file_path = os.path.join(sound_folder, filename)
    word = filename.split(".")[0]  # Extract the word from the filename
    # Removes the number and _ from the word Ex. 1_hello.wav -> hello
    word = re.sub(r'[^A-Za-z]+', '', word)
    if word not in words:
        words.append(word)
    
    label.append(words.index(word))
    name.append(word)

    audio_data, sample_rate = librosa.load(file_path, sr=None)  # Load audio file
    sound_data.append(audio_data)

In [779]:
label

[0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 4,
 3,
 5,
 6,
 7,
 6,
 8,
 9,
 10,
 11,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0]

In [780]:
name

['Naam',
 'Naam',
 'Saurabh',
 'Saurabh',
 'Saurabh',
 'Saurabh',
 'Baral',
 'Baral',
 'Baral',
 'ho',
 'Miro',
 'ho',
 'tapai',
 'Ko',
 'tapay',
 'Ko',
 'xa',
 'I',
 'love',
 'you',
 'Miro',
 'Miro',
 'Miro',
 'Miro',
 'Naam',
 'Naam',
 'Naam',
 'Naam']

In [781]:
words

['Naam',
 'Saurabh',
 'Baral',
 'ho',
 'Miro',
 'tapai',
 'Ko',
 'tapay',
 'xa',
 'I',
 'love',
 'you']

In [782]:
words.index(word)

0

In [783]:
os.listdir(sound_folder)

['10_Naam.wav',
 '11_Naam.wav',
 '12_Saurabh.wav',
 '13_Saurabh.wav',
 '14_Saurabh.wav',
 '15_Saurabh.wav',
 '16_Baral.wav',
 '17_Baral.wav',
 '18_Baral.wav',
 '19_ho.wav',
 '1_Miro.wav',
 '20_ho.wav',
 '21_tapai.wav',
 '22_Ko.wav',
 '23_tapay.wav',
 '24_Ko.wav',
 '25_xa.wav',
 '26_I.wav',
 '27_love.wav',
 '28_you.wav',
 '2_Miro.wav',
 '3_Miro.wav',
 '4_Miro.wav',
 '5_Miro.wav',
 '6_Naam.wav',
 '7_Naam.wav',
 '8_Naam.wav',
 '9_Naam.wav']

In [784]:
label

[0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 4,
 3,
 5,
 6,
 7,
 6,
 8,
 9,
 10,
 11,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0]

In [785]:
words

['Naam',
 'Saurabh',
 'Baral',
 'ho',
 'Miro',
 'tapai',
 'Ko',
 'tapay',
 'xa',
 'I',
 'love',
 'you']

In [786]:
sample_rate

48000

In [787]:
words, len(sound_data)

(['Naam',
  'Saurabh',
  'Baral',
  'ho',
  'Miro',
  'tapai',
  'Ko',
  'tapay',
  'xa',
  'I',
  'love',
  'you'],
 28)

In [788]:
# Step 2: Preprocessing
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(words)
sound_data = [librosa.feature.mfcc(y=data, sr=sample_rate) for data in sound_data]

In [789]:
name = label_encoder.fit_transform(name)
name

array([ 4,  4,  5,  5,  5,  5,  0,  0,  0,  6,  3,  6,  8,  2,  9,  2, 10,
        1,  7, 11,  3,  3,  3,  3,  4,  4,  4,  4], dtype=int64)

In [790]:
labels

array([ 4,  5,  0,  6,  3,  8,  2,  9, 10,  1,  7, 11], dtype=int64)

In [791]:
labels

array([ 4,  5,  0,  6,  3,  8,  2,  9, 10,  1,  7, 11], dtype=int64)

In [792]:
label_encoder.classes_

array(['Baral', 'I', 'Ko', 'Miro', 'Naam', 'Saurabh', 'ho', 'love',
       'tapai', 'tapay', 'xa', 'you'], dtype='<U7')

In [793]:
labels = utils.to_categorical(labels, num_classes=len(label_encoder.classes_))


In [794]:
labels

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [795]:
label = utils.to_categorical(label, num_classes=len(label_encoder.classes_))
print(label.shape)
label

(28, 12)


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.,

In [796]:
labels.shape

(12, 12)

In [797]:
len(sound_data)

28

In [798]:
sound_data[0].shape

(20, 126)

In [799]:
# Split the dataset into training and testing sets

In [800]:
import numpy as np

# Determine the maximum length of the arrays in x_train
max_length = max(data.shape[1] for data in sound_data)

# Pad the sequences to the maximum length
padded_x_train = []
for data in sound_data:
    if data.shape[1] < max_length:
        padding = np.zeros((data.shape[0], max_length - data.shape[1]))
        padded_data = np.concatenate([data, padding], axis=1)
    else:
        padded_data = data
    padded_x_train.append(padded_data)

# Convert padded_x_train to a NumPy array
padded_x_train = np.array(padded_x_train)

# Print the padded sequences
print(padded_x_train)


[[[-2.99812622e+02 -2.68945435e+02 -2.52896881e+02 ... -2.60914032e+02
   -2.64518402e+02 -2.22407089e+02]
  [ 1.78257141e+02  2.12234970e+02  2.32832626e+02 ...  2.45919907e+02
    2.43404022e+02  2.03795563e+02]
  [ 6.77690220e+00 -2.28766918e+01 -2.92725182e+01 ... -5.17024326e+00
    2.25792122e+00  3.54791565e+01]
  ...
  [-6.11440277e+00 -9.37044716e+00 -1.29157467e+01 ...  1.59050226e-02
   -9.85843420e-01  2.01994514e+00]
  [-1.17798119e+01 -1.04930725e+01 -4.18545485e+00 ...  3.72642446e+00
    6.65921402e+00  5.80807257e+00]
  [-5.29475069e+00 -4.86009884e+00 -4.92752504e+00 ... -1.08880687e+00
   -2.02197981e+00  4.78213596e+00]]

 [[-2.23669174e+02 -2.45219788e+02 -2.68495178e+02 ... -2.28076248e+02
   -2.23399124e+02 -2.24804199e+02]
  [ 1.91913513e+02  2.09324524e+02  2.32184387e+02 ...  2.61799927e+02
    2.63875793e+02  2.66698914e+02]
  [ 3.92412262e+01  2.60527668e+01  7.46072340e+00 ... -5.39751005e+00
   -4.00017071e+00  7.70471931e-01]
  ...
  [-1.29654717e+00 -4.0

In [801]:
sound_data = padded_x_train

In [802]:
sound_data.shape

(28, 20, 126)

In [803]:
sound_data

array([[[-2.99812622e+02, -2.68945435e+02, -2.52896881e+02, ...,
         -2.60914032e+02, -2.64518402e+02, -2.22407089e+02],
        [ 1.78257141e+02,  2.12234970e+02,  2.32832626e+02, ...,
          2.45919907e+02,  2.43404022e+02,  2.03795563e+02],
        [ 6.77690220e+00, -2.28766918e+01, -2.92725182e+01, ...,
         -5.17024326e+00,  2.25792122e+00,  3.54791565e+01],
        ...,
        [-6.11440277e+00, -9.37044716e+00, -1.29157467e+01, ...,
          1.59050226e-02, -9.85843420e-01,  2.01994514e+00],
        [-1.17798119e+01, -1.04930725e+01, -4.18545485e+00, ...,
          3.72642446e+00,  6.65921402e+00,  5.80807257e+00],
        [-5.29475069e+00, -4.86009884e+00, -4.92752504e+00, ...,
         -1.08880687e+00, -2.02197981e+00,  4.78213596e+00]],

       [[-2.23669174e+02, -2.45219788e+02, -2.68495178e+02, ...,
         -2.28076248e+02, -2.23399124e+02, -2.24804199e+02],
        [ 1.91913513e+02,  2.09324524e+02,  2.32184387e+02, ...,
          2.61799927e+02,  2.63875793e

In [804]:
# new_label = utils.to_categorical(labels, num_classes=len(label_encoder.classes_))
# new_label

In [805]:
labels

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [806]:
sound_data.shape

(28, 20, 126)

In [807]:
labels.shape

(12, 12)

In [808]:
label = np.array(label)

In [809]:
x_train, x_test, y_train, y_test = train_test_split(sound_data, label, test_size=0.2, random_state=42)


In [810]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((22, 20, 126), (6, 20, 126), (22, 12), (6, 12))

In [811]:
x_train.shape

(22, 20, 126)

In [812]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [813]:
padded_x_train[1].shape

(20, 126)

In [814]:
# y_train = utils.to_categorical(y_train, num_classes=len(label_encoder.classes_))
# y_test = utils.to_categorical(y_test, num_classes=len(label_encoder.classes_))

In [815]:
y_train, y_test

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0

In [816]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((22, 20, 126), (6, 20, 126), (22, 12), (6, 12))

In [817]:
# Step 3: Model Training
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))


In [818]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [819]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 18, 124, 32)       320       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 9, 62, 32)        0         
 g2D)                                                            
                                                                 
 flatten_16 (Flatten)        (None, 17856)             0         
                                                                 
 dense_32 (Dense)            (None, 64)                1142848   
                                                                 
 dense_33 (Dense)            (None, 12)                780       
                                                                 
Total params: 1,143,948
Trainable params: 1,143,948
Non-trainable params: 0
___________________________________________

In [820]:
x_train[0].shape, x_train[1].shape, x_train[2].shape


((20, 126, 1), (20, 126, 1), (20, 126, 1))

In [821]:
# y_train = utils.to_categorical(y_train, num_classes=len(label_encoder.classes_))
# y_test = utils.to_categorical(y_test, num_classes=len(label_encoder.classes_))

In [822]:
labels

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [823]:
np.expand_dims(x_train, axis=-1).shape, y_train.shape, np.expand_dims(x_test, axis=-1).shape, y_test.shape

((22, 20, 126, 1, 1), (22, 12), (6, 20, 126, 1, 1), (6, 12))

In [824]:
y_train.shape, y_test.shape

((22, 12), (6, 12))

In [825]:
x_train.shape

(22, 20, 126, 1)

In [826]:
np.expand_dims(x_train, axis=-1).shape

(22, 20, 126, 1, 1)

In [827]:
y_train.shape

(22, 12)

In [828]:
y_train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.,

In [829]:
y_test

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [830]:
x_train.shape

(22, 20, 126, 1)

In [831]:
y_train.shape

(22, 12)

In [832]:
x_train.shape

(22, 20, 126, 1)

In [833]:
y_train.shape

(22, 12)

In [834]:
y_train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.,

In [835]:
# y_train = utils.to_categorical(y_train, num_classes=len(label_encoder.classes_))
# y_test = utils.to_categorical(y_test, num_classes=len(label_encoder.classes_))


In [836]:
y_train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.,

In [837]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10


1/1 [==============================] - 4s 4s/step - loss: 38.8889 - accuracy: 0.0455 - val_loss: 118.7515 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 129ms/step - loss: 106.3537 - accuracy: 0.1818 - val_loss: 106.8027 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 141ms/step - loss: 138.6534 - accuracy: 0.0455 - val_loss: 86.9121 - val_accuracy: 0.1667
Epoch 4/10
1/1 [==============================] - 0s 136ms/step - loss: 129.6525 - accuracy: 0.0000e+00 - val_loss: 81.9878 - val_accuracy: 0.1667
Epoch 5/10
1/1 [==============================] - 0s 118ms/step - loss: 105.1334 - accuracy: 0.2727 - val_loss: 86.5837 - val_accuracy: 0.1667
Epoch 6/10
1/1 [==============================] - 0s 132ms/step - loss: 99.5761 - accuracy: 0.2727 - val_loss: 80.4081 - val_accuracy: 0.1667
Epoch 7/10
1/1 [==============================] - 0s 119ms/step - loss: 90.7290 - accuracy: 0.1364 - val_loss: 61.8637 - val_accuracy: 0.1667
Ep

In [838]:
y_pred = model.predict(np.expand_dims(x_test, axis=-1))

1/1 [==============================] - 0s 137ms/step


In [839]:
y_pred

array([[9.9783957e-01, 3.2501538e-10, 2.7195517e-27, 1.3702242e-18,
        2.1603988e-03, 0.0000000e+00, 7.0258815e-22, 0.0000000e+00,
        7.6300118e-15, 0.0000000e+00, 1.5283433e-35, 0.0000000e+00],
       [9.9993777e-01, 1.1381518e-13, 1.5870949e-30, 4.2865170e-21,
        6.2257714e-05, 0.0000000e+00, 2.7529118e-31, 0.0000000e+00,
        3.9406466e-14, 0.0000000e+00, 3.9294004e-38, 0.0000000e+00],
       [6.4510596e-01, 2.5661222e-09, 1.4863179e-26, 2.8281091e-19,
        3.5489407e-01, 0.0000000e+00, 7.5281966e-24, 0.0000000e+00,
        3.8574561e-16, 0.0000000e+00, 6.7408388e-34, 0.0000000e+00],
       [1.1442095e-03, 3.6879944e-17, 2.0014139e-32, 4.3931038e-25,
        9.9885583e-01, 0.0000000e+00, 3.5743630e-32, 0.0000000e+00,
        1.7542305e-15, 0.0000000e+00, 5.6000887e-36, 0.0000000e+00],
       [9.9999976e-01, 1.5665240e-09, 2.6558659e-25, 5.3226911e-20,
        2.2569932e-07, 0.0000000e+00, 2.1311906e-27, 0.0000000e+00,
        1.0975190e-14, 0.0000000e+00, 0.0000

In [840]:
y_test

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [841]:
# import accuracy_score
y_test, y_train

(array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

In [842]:
new_sound_file = "path_to_new_sound_file.wav"
new_sound_data, sample_rate = librosa.load(new_sound_file, sr=None)
new_sound_data = librosa.feature.mfcc(y=new_sound_data, sr=sample_rate)
new_sound_data = np.expand_dims(new_sound_data, axis=0)
new_sound_data = np.expand_dims(new_sound_data, axis=-1)

C:\Users\baral\AppData\Local\Temp\ipykernel_872\484566982.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  new_sound_data, sample_rate = librosa.load(new_sound_file, sr=None)
c:\Users\baral\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_new_sound_file.wav'